In [1]:
print("Ok")

Ok


In [2]:
%pwd


'c:\\Users\\Dell\\Downloads\\End-to-end-Medical-Chatbot-Generative-AI-main\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Dell\\Downloads\\End-to-end-Medical-Chatbot-Generative-AI-main'

In [6]:
pip install langchain pypdf


  Using cached langchain-0.3.23-py3-none-any.whl.metadata (7.8 kB)
Using cached langchain-0.3.23-py3-none-any.whl (1.0 MB)
Note: you may need to restart the kernel to use updated packages.


In [12]:
!pip install langchain-community[pdf]
!pip install pypdf



  Using cached langchain_community-0.3.21-py3-none-any.whl.metadata (2.4 kB)
Using cached langchain_community-0.3.21-py3-none-any.whl (2.5 MB)


In [13]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter



In [14]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [15]:
os.makedirs("Data/", exist_ok=True)


In [18]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [19]:
def load_pdf_file(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents


In [20]:
extracted_data = load_pdf_file(data="C:/Users/Dell/Downloads/End-to-end-Medical-Chatbot-Generative-AI-main/Data/")


In [21]:
extracted_data=load_pdf_file(data='Data/')

In [22]:
# extracted_data

In [23]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [24]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [25]:
# text_chunks

In [26]:
from langchain.embeddings import HuggingFaceEmbeddings

In [27]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [28]:
embeddings = download_hugging_face_embeddings()

C:\Users\Dell\AppData\Local\Temp\ipykernel_6592\1196424635.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\Dell\anaconda3\envs\medibot\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\Dell\anaconda3\envs\medibot\lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please

In [29]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [30]:
# query_result

In [31]:
from dotenv import load_dotenv
load_dotenv()

True

In [33]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
GROQ_API_KEY = os.environ.get('GROQ_API_KEY')

In [34]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")


In [35]:
print(PINECONE_API_KEY)  # Should not be None or empty


pcsk_3QG9Tt_SJtA1whjfa4qmvYL1bsXyN4nWvdSKmNK7YGetjfXAu5iPrX5T8Wi4Xu2GvkLhCh


In [37]:
from pinecone import Pinecone


In [39]:
PINECONE_API_KEY = "pcsk_3QG9Tt_SJtA1whjfa4qmvYL1bsXyN4nWvdSKmNK7YGetjfXAu5iPrX5T8Wi4Xu2GvkLhCh"
pc = Pinecone(api_key=PINECONE_API_KEY)


In [40]:
pc = Pinecone(api_key=PINECONE_API_KEY, environment="medicalbot")


In [41]:
print(pc.list_indexes())  



{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 384,
              'host': 'medicalbot-4vv0g14.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'medicalbot',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}


In [46]:
# No need to call `create_index`, just connect to the existing one:
index = pc.Index("medicalbot")


In [50]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [51]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [52]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [53]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [54]:
docsearch

In [55]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [56]:
retrieved_docs = retriever.invoke("What is Acne?")

In [57]:
retrieved_docs

[Document(id='ca3fd4e5-0ee4-41f0-a110-07de874c9d2d', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='a4941544-303f-4a43-8415-49601df554be', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM -

In [58]:
pip install groq


Note: you may need to restart the kernel to use updated packages.


In [59]:
pip install langchain-groq


Note: you may need to restart the kernel to use updated packages.


In [61]:
!pip install langchain-groq langchain-community



In [74]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0.4,
    model_name="llama3-8b-8192",  # or another model Groq supports
    api_key=os.environ["GROQ_API_KEY"]
)



In [75]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [76]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [77]:
from groq import Groq

client = Groq(api_key="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")

response = client.chat.completions.create(
    model="llama3-8b-8192",  # or llama3-8b-8192, gemma-7b-it, etc.
    messages=[
        {"role": "user", "content": "What is Acromegaly and gigantism?"}
    ],
    temperature=0.4,
    max_tokens=100
)

print(response.choices[0].message.content)



Acromegaly and gigantism are two related conditions that result from excessive growth hormone (GH) production.

**Acromegaly:**

Acromegaly is a rare hormonal disorder that occurs when the pituitary gland produces too much growth hormone (GH) in adults. This excess GH causes the bones, muscles, and other tissues to grow excessively, leading to a range of symptoms. Acromegaly typically develops in adulthood, usually between the ages of 30 and 50


In [78]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-8b-8192",  # ✅ changed from mixtral-8x7b-32768
    api_key="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"
)


In [79]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])


I don't know. The provided context does not mention "stats". It appears to be discussing a complete blood count (CBC) and athletic heart syndrome.
